# 한국어 라우팅 벤치: TF‑IDF와 gensim Word2Vec 학습 및 평가

이 노트북은 다음을 포함합니다.

1. 데이터 로드, 학습/검증 분할
2. TF‑IDF 기반 분류기 학습과 평가
3. gensim Word2Vec로 문서 임베딩 생성 후 분류기 학습과 평가
4. 사용자 제공 휴리스틱 라우터 코드로 비교 평가 및 시각화

주의: 이 환경에는 `gensim`이 없을 수 있습니다. 없으면 Word2Vec 셀은 건너뜁니다.

In [1]:

# 환경 설정과 데이터 로드
import re, random, json, types
import numpy as np
import pandas as pd
from pathlib import Path

CSV_PATH = Path("train_koen_router_bench.csv")
assert CSV_PATH.exists(), f"CSV not found at {CSV_PATH}"

df = pd.read_csv(CSV_PATH)
print("Loaded shape:", df.shape)
print(df.head())


Loaded shape: (14703, 5)
                                            question subject difficulty  \
0  On $\triangle ABC$ points $A,D,E$, and $B$ lie...    math       hard   
1  There are $8!=40320$ eight-digit positive inte...    math       hard   
2  The twelve letters $A,B,C,D,E,F,G,H,I,J,K$, an...    math       hard   
3  The parabola with equation $y=x^{2}-4$ is rota...    math       hard   
4  The set of points in 3-dimensional coordinate ...    math       hard   

  language route_rule  
0       en         M7  
1       en         M7  
2       en         M7  
3       en         M7  
4       en         M7  


In [2]:

# 텍스트/라벨 컬럼 자동 감지 및 분할
from sklearn.model_selection import train_test_split

text_col = "question" if "question" in df.columns else df.columns[0]
label_col = "route_rule" if "route_rule" in df.columns else df.columns[-1]
print("Using columns -> text:", text_col, "label:", label_col)

df = df[[text_col, label_col]].dropna().reset_index(drop=True)
X = df[text_col].astype(str).values
y = df[label_col].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
len(X_train), len(X_test)


Using columns -> text: question label: route_rule


(11762, 2941)

## TF‑IDF 분류기

In [3]:

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

# 1) char ngram 기반
tfidf_char = Pipeline([
    ("tfidf", TfidfVectorizer(analyzer="char", ngram_range=(2,5), min_df=3)),
    ("clf", LinearSVC())
])
tfidf_char.fit(X_train, y_train)
pred_char = tfidf_char.predict(X_test)
acc_char = accuracy_score(y_test, pred_char)
print("TF‑IDF char Accuracy:", acc_char)
print(classification_report(y_test, pred_char))

# 2) word 기반
tfidf_word = Pipeline([
    ("tfidf", TfidfVectorizer(analyzer="word", token_pattern=r"(?u)\b\w+\b", min_df=3)),
    ("clf", LinearSVC())
])
tfidf_word.fit(X_train, y_train)
pred_word = tfidf_word.predict(X_test)
acc_word = accuracy_score(y_test, pred_word)
print("TF‑IDF word Accuracy:", acc_word)
print(classification_report(y_test, pred_word))


TF‑IDF char Accuracy: 0.99081944916695
              precision    recall  f1-score   support

         B13       0.99      0.92      0.95       254
          K7       1.00      1.00      1.00       572
          M7       0.99      0.99      0.99       159
         WB7       0.99      1.00      0.99      1956

    accuracy                           0.99      2941
   macro avg       0.99      0.98      0.98      2941
weighted avg       0.99      0.99      0.99      2941

TF‑IDF word Accuracy: 0.9806188371302278
              precision    recall  f1-score   support

         B13       0.96      0.83      0.89       254
          K7       1.00      0.99      1.00       572
          M7       0.99      0.97      0.98       159
         WB7       0.98      1.00      0.99      1956

    accuracy                           0.98      2941
   macro avg       0.98      0.95      0.96      2941
weighted avg       0.98      0.98      0.98      2941



## gensim Word2Vec 분류기
train 세트로만 Word2Vec을 학습하고, 문서 임베딩은 단어 벡터 평균으로 만듭니다.

In [4]:

import re
import numpy as np

from gensim.models import Word2Vec

def simple_tokenize(text: str):
    text = re.sub(r"[^가-힣A-Za-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text.split()


train_tokens = [simple_tokenize(t) for t in X_train]
test_tokens  = [simple_tokenize(t) for t in X_test]

w2v = Word2Vec(
    sentences=train_tokens,
    vector_size=100,
    window=8,
    min_count=5,
    workers=4,
    sg=0,
    epochs=5
)
def doc_vec(tokens, model):
    vecs = [model.wv[w] for w in tokens if w in model.wv]
    if not vecs:
        return np.zeros(model.vector_size, dtype=np.float32)
    return np.mean(vecs, axis=0)

Xtr_w2v = np.vstack([doc_vec(t, w2v) for t in train_tokens])
Xte_w2v = np.vstack([doc_vec(t, w2v) for t in test_tokens])

from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report
clf_w2v = LinearSVC()
clf_w2v.fit(Xtr_w2v, y_train)
pred_w2v = clf_w2v.predict(Xte_w2v)
acc_w2v = accuracy_score(y_test, pred_w2v)
print("Word2Vec LinearSVC Accuracy:", acc_w2v)
print(classification_report(y_test, pred_w2v))



Word2Vec LinearSVC Accuracy: 0.9666780006800408
              precision    recall  f1-score   support

         B13       0.90      0.78      0.83       254
          K7       0.99      0.98      0.99       572
          M7       0.96      0.95      0.96       159
         WB7       0.97      0.99      0.98      1956

    accuracy                           0.97      2941
   macro avg       0.95      0.92      0.94      2941
weighted avg       0.97      0.97      0.97      2941



## test 셋에서 평가

In [6]:
test = pd.read_csv("test_koen_router_bench_w_oracle.csv")

In [8]:
test

,question,subject,difficulty,language,route_rule
0,Define pure competition.\n1) Pure competition ...,general,medium,en,WB7
1,3주 전 자연분만한 37세 산모가 기운이 없고 땀이 많아서 병원에 왔다. 가만히 있...,general,medium,ko,K7
2,When a block of wood of mass 1 kg is held in a...,general,medium,en,WB7
3,Oxygenated blood is carried to the heart by wh...,general,medium,en,WB7
4,A landowner owned a large building in the city...,general,medium,en,WB7
...,...,...,...,...,...
2495,"A chemist possessesKCl, PH_3, GeCl_4, H_2S, an...",general,medium,en,WB7
2496,"\(\iint_D \frac{|y|}{\sqrt{(x-2)^2+y^2}}dxdy, ...",math,hard,ko,M7
2497,An epidemic involving 10 individuals of all ag...,general,medium,en,WB7
2498,An American buys an entertainment system that ...,general,medium,en,WB7


In [14]:
y_pred_char = tfidf_char.predict(test['question'].astype(str).values)
print("\n[TF‑IDF char] Accuracy:", accuracy_score(test['route_rule'], y_pred_char))
print(classification_report(test['route_rule'], y_pred_char))


[TF‑IDF char] Accuracy: 0.9852
              precision    recall  f1-score   support

         B13       0.97      0.91      0.94       255
          K7       0.99      1.00      1.00       555
          M7       1.00      0.95      0.98       190
         WB7       0.98      1.00      0.99      1500

    accuracy                           0.99      2500
   macro avg       0.99      0.96      0.97      2500
weighted avg       0.99      0.99      0.98      2500



In [16]:
y_pred_word = tfidf_word.predict(test['question'].astype(str).values)
print("\n[TF‑IDF word] Accuracy:", accuracy_score(test['route_rule'], y_pred_word))
print(classification_report(test['route_rule'], y_pred_word))


[TF‑IDF word] Accuracy: 0.9712
              precision    recall  f1-score   support

         B13       0.95      0.79      0.86       255
          K7       1.00      0.99      0.99       555
          M7       1.00      0.95      0.97       190
         WB7       0.96      1.00      0.98      1500

    accuracy                           0.97      2500
   macro avg       0.98      0.93      0.95      2500
weighted avg       0.97      0.97      0.97      2500



In [18]:
test_tokens = [ simple_tokenize(t) for t in test['question'].astype(str).values ]
def _doc_vec(tokens, model):
    vecs = [model.wv[w] for w in tokens if w in model.wv]
    if not vecs:
        return np.zeros(model.vector_size, dtype=np.float32)
    return np.mean(vecs, axis=0)
Xte_w2v_ext = np.vstack([_doc_vec(t, w2v) for t in test_tokens])
y_pred_w2v_ext = clf_w2v.predict(Xte_w2v_ext)
print("\n[Word2Vec] Accuracy:", accuracy_score(test['route_rule'], y_pred_w2v_ext))
print(classification_report(test['route_rule'], y_pred_w2v_ext))


[Word2Vec] Accuracy: 0.9584
              precision    recall  f1-score   support

         B13       0.89      0.77      0.83       255
          K7       0.99      0.98      0.99       555
          M7       0.98      0.92      0.95       190
         WB7       0.96      0.99      0.97      1500

    accuracy                           0.96      2500
   macro avg       0.95      0.91      0.93      2500
weighted avg       0.96      0.96      0.96      2500



## LiteLLM과 연결

In [24]:
mmap = {
    "B13": "openrouter/qwen/qwen3-14b:free",
    "K7": "openrouter/google/gemma-3-12b-it:free",
    "M7": "openrouter/meta-llama/llama-3.3-8b-instruct:free",
    "WB7": "openrouter/google/gemma-3n-e4b-it:free"
}

question = "씨 감자를 심으려 한다. 씨감자의 무게가 80g이라면 몇 조각으로 자르는 것이 가장 좋은가?"
pred_model = tfidf_word.predict([question])[0]

or_model = mmap[pred_model]

In [27]:
from litellm import completion
import os

## set ENV variables
os.environ["OPENROUTER_API_KEY"] = "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"

response = completion(
  model=or_model,
  messages=[{ "content": question,"role": "user"}]
)

In [30]:
print(response.choices[0].message.content)

씨감자를 몇 조각으로 자르는 것이 가장 좋을지는 여러 요인에 따라 달라집니다. 일반적으로 다음과 같은 사항들을 고려해야 합니다.

**1. 감자 품종:**

*   **대형 품종:** 덩이뿌리가 큰 품종은 2~3개 조각으로 나누는 것이 좋습니다.
*   **소형 품종:** 덩이뿌리가 작은 품종은 1개 조각으로 심거나, 2개 조각으로 나누어도 괜찮습니다.

**2. 씨감자 무게:**

*   80g 씨감자는 일반적으로 1~3개 조각으로 나누어 심습니다.
*   **1개 조각:** 씨감자 무게가 80g이라면, 1개 조각으로 심는 것이 가장 좋습니다.
*   **2개 조각:** 씨감자를 2개 조각으로 나누려면, 각 조각의 무게가 40g 정도가 되도록 자릅니다.
*   **3개 조각:** 씨감자를 3개 조각으로 나누려면, 각 조각의 무게가 약 27g 정도가 되도록 자릅니다.

**3. 눈 (싹):**

*   각 조각에 눈이 1~3개 정도 포함되도록 자르는 것이 좋습니다. 눈은 싹이 트는 부분으로, 눈이 없으면 발아가 되지 않습니다.
*   너무 많은 눈이 한 조각에 있으면, 싹이 여러 방향으로 나와 생육이 약해질 수 있습니다.

**4. 상처:**

*   씨감자를 자를 때 상처가 나지 않도록 주의해야 합니다. 상처 부위는 병원균에 감염될 가능성이 높습니다.
*   상처가 났을 경우, 톱밥이나 나무가루 등으로 상처 부위를 덮어주면 감염을 예방할 수 있습니다.

**일반적인 권장 사항:**

*   80g 씨감자는 **1개 조각으로 심는 것이 가장 좋습니다.**
*   만약 2개 조각으로 나누어 심을 경우, 각 조각의 무게가 40g 정도가 되도록 하고, 각 조각에 눈이 1~2개 포함되도록 합니다.

**참고:**

*   씨감자 품종, 재배 환경, 농가의 경험 등에 따라 적절한 조각 개수가 달라질 수 있습니다.
*   씨감자 조각을 심기 전에 1~2주 정도 따뜻하고 밝은 곳에 보관하여 발아를 촉진하는 것이 좋습니다.

**추가 정보:**

*   씨감자 심는 방법: 